# 02: Training...


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from rxwgan.wgan import wgan_optimizer
import tensorflow as tf
from sklearn.model_selection import KFold
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [2]:
tb_image_path = '/home/guilherme.dionisio/bric_data/Shenzhen/raw'

In [ ]:
!pip install atlas_mpl_style

In [3]:
import atlas_mpl_style as ampl
ampl.use_atlas_style()

## Create my Models:

In [4]:
from rxwgan.models import Model_v1 as Model
gan = Model(trainable=True)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense (Dense)                (None, 8192)              827392    
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 64)        32832     
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)       

## Preprocess Data

In [12]:
dataframe = pd.read_csv(tb_image_path + '/'+'Shenzhen_table_from_raw.csv')
dataframe = dataframe[dataframe['target'] == False]

kf = KFold(n_splits = 5, shuffle = True, random_state = 512)

#dataframe.head()
#dataframe.shape[0]

train_datagen = ImageDataGenerator( rescale=1./255 )
batch_size = 16
for train_idx, val_idx in kf.split(dataframe):
    #print(len(train_idx)+len(val_idx))
    training_data = dataframe.iloc[train_idx]
    validation_data = dataframe.iloc[val_idx]
    train_data_generator = train_datagen.flow_from_dataframe(training_data, directory = None,
                                                             x_col = 'raw_image_path', batch_size = batch_size,
                                                             target_size = (gan.height,gan.width), 
                                                             class_mode = None, shuffle = True,
                                                             validate_filenames = False, color_mode = 'grayscale')
    valid_data_generator = train_datagen.flow_from_dataframe(validation_data, directory = None,
                                                             x_col = 'raw_image_path', batch_size = batch_size,
                                                             target_size = (gan.height,gan.width),
                                                             class_mode = None, shuffle = True,
                                                             validate_filenames = False, color_mode = 'grayscale')


Found 260 non-validated image filenames.
Found 66 non-validated image filenames.
Found 261 non-validated image filenames.
Found 65 non-validated image filenames.
Found 261 non-validated image filenames.
Found 65 non-validated image filenames.
Found 261 non-validated image filenames.
Found 65 non-validated image filenames.
Found 261 non-validated image filenames.
Found 65 non-validated image filenames.


## Train Model:

Reshape each image to 128 X 128 and rescale all to gray (1/255). Reading all from tuberculoses rx images

In [ ]:
batch_size=16

train_datagen = ImageDataGenerator( rescale=1./255 )
train_generator = train_datagen.flow_from_directory( tb_image_path, 
                                                     color_mode='grayscale', 
                                                     target_size=(gan.height,gan.width), 
                                                     batch_size=batch_size,
                                                     classes=['no_tb'] 
                                                     )

optimizer = wgan_optimizer( gan, n_discr=0, max_epochs=1000, disp_for_each = 50, save_for_each=50 )
history = optimizer.train( train_generator )



In [ ]:
optimizer.plot_evolution(history, 'output/evolution.pdf')